In [1]:
from urllib import request
import pandas as pd

In [2]:
URLS = {"Confirmed":
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv', 
        "Recovered":
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv',
        "Deaths":
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv', 
        }

for url_key in URLS:
    print ("- Downloading", url_key, "...")
    request.urlretrieve(URLS[url_key], './input/{}_cases.csv'.format(url_key))
print ("- Done!")


confirmed = pd.read_csv('./input/Confirmed_cases.csv')
recovered  =recv_df = pd.read_csv('./input/Recovered_cases.csv')
deaths = pd.read_csv('./input/Deaths_cases.csv')

confirmed.head()

- Downloading Confirmed ...
- Downloading Recovered ...
- Downloading Deaths ...
- Done!


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,75,82,114,147,177,212,272,322,411,599
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,701,773,839,825,878,889,924,963,1007,1086
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,200,212,226,243,266,313,345,385,432,455
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,2
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,197,238,428,566,673,790,900,1030,1183,1306


In [3]:
# Getting all dates
all_dates = confirmed.columns[4:]

new_confirmed = confirmed.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=all_dates, var_name='Date', value_name='Confirmed')

new_recovered = recovered.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=all_dates, var_name='Date', value_name='Recovered')

new_deaths = deaths.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=all_dates, var_name='Date', value_name='Deaths')

clean_data = pd.concat([new_confirmed, new_recovered['Recovered'], new_deaths['Deaths']], axis=1)


# removing county wise data to avoid double counting
# clean_data = clean_data[clean_data['Province/State'].str.contains(',')!=True]

clean_data.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Recovered,Deaths
0,NaN,Thailand,15.0000,101.0000,1/22/20,2,0,0
1,NaN,Japan,36.0000,138.0000,1/22/20,2,0,0
2,NaN,Singapore,1.2833,103.8333,1/22/20,0,0,0
3,NaN,Nepal,28.1667,84.2500,1/22/20,0,0,0
4,NaN,Malaysia,2.5000,112.5000,1/22/20,0,0,0


In [4]:
clean_data.rename(columns={'Country/Region': 'Country'}, inplace=True)

In [5]:
clean_data

,Province/State,Country,Lat,Long,Date,Confirmed,Recovered,Deaths
0,NaN,Thailand,15.0000,101.0000,1/22/20,2,0,0
1,NaN,Japan,36.0000,138.0000,1/22/20,2,0,0
2,NaN,Singapore,1.2833,103.8333,1/22/20,0,0,0
3,NaN,Nepal,28.1667,84.2500,1/22/20,0,0,0
4,NaN,Malaysia,2.5000,112.5000,1/22/20,0,0,0
...,...,...,...,...,...,...,...,...
29702,NaN,Dominica,15.4150,-61.3710,3/22/20,1,0,0
29703,NaN,Grenada,12.1165,-61.6790,3/22/20,1,0,0
29704,NaN,Mozambique,-18.6657,35.5296,3/22/20,1,0,0
29705,NaN,Syria,34.8021,38.9968,3/22/20,1,0,0


In [6]:
clean_data = clean_data.groupby(["Country", "Date"])[['Confirmed', 'Deaths', 'Recovered']].sum().reset_index()

In [7]:
clean_data["Country"].replace({'US': 'United States'}, inplace=True)
clean_data["Country"].replace({'Czechia': 'Czech Republic'}, inplace=True)
clean_data["Country"].replace({'Korea, South': 'South Korea'}, inplace=True)

In [8]:
clean_data.to_csv("./input/covid_19_clear.csv", index=False)